In [1]:
import pandas as pd
import os
from datetime import datetime

In [2]:
from config.config_loader import Config

config = Config()

In [3]:
db_elec_path = config.get_path("data_path.elec.processed.facturas")
db_elec_manual_path = config.get_resolved_path("data_path.elec.processed.facturas_manual")

path_alias_elec = config.get_resolved_path("data_path.elec.customers.alias_elec")

path_raw_data_elec = config.get_resolved_path("data_path.elec.invoice_comp.data_raw")
customers_path_single = config.get_resolved_path("data_path.elec.customers.folder")
path_fras_elec_xml_clean = config.get_resolved_path("data_path.elec.invoice_comp.xml_folder_clean")
customers_path_single

'c:\\Users\\mikel.perez\\0_Python_projects_v2\\data\\customers\\elec'

In [4]:
from invoice_comp.check_utility import data_extraction
from utils.utils_var import insert_alias

In [5]:
clientes_elec = os.listdir(customers_path_single)
clientes_elec

['alias_clientes_elec - copia.xlsx',
 'alias_clientes_elec.xlsx',
 'Altan Pharma',
 'Catalonia Fundacio',
 'Generic',
 'Grupo Inspired',
 'Kem One',
 'Montfrisa',
 'PR',
 'PR2',
 'Sundolitt',
 'Taghleef',
 'Trilla y Famesa',
 'VYOSA']

In [6]:
path_facturas_cliente = f'{customers_path_single}/PR/facturas/'
print((os.listdir(path_facturas_cliente)))

df_keys = pd.DataFrame()

for yr in os.listdir(path_facturas_cliente):

    months_path = (f'{path_facturas_cliente}/{yr}')

    # print(months_path)

    for month_single in os.listdir(months_path):
        list_fras = []
        print(f'{yr}/{month_single}')
        fras_list = (os.listdir(f'{months_path}/{month_single}'))

        for fra in fras_list:

            if '.pdf' in fra:

                list_fras.append(int(fra[:-4]))

            fras_df = pd.DataFrame(list_fras)
            fras_df['year'] = int(yr)
            fras_df['month'] = (month_single)
            fras_df.columns = ['fra_ref', 'year', 'month']
            fras_df
            fras_df = fras_df[['year', 'month', 'fra_ref']]
            fras_df.columns = ['Año', 'Mes', 'Referencia Factura']
    
        if not df_keys.empty:
            df_keys = pd.concat([df_keys, fras_df], ignore_index=True)
        else:
            df_keys = fras_df.copy()

df_keys_fil = df_keys[df_keys['Año'].isin([2024,2025])]
df_keys_fil.to_excel('Referencias facturas electricidad.xlsx', index=False)
df_keys_fil

# las facturas se vuelcan en data/invoice_comp/facturas/PR, organizadas por fecha de emisión
# hay que ir a data/customers/elec/PR/facturas y crear la carpeta de mes de suministro (mes de emision - 1) y pegar las facturas

['2022', '2023', '2024', '2025']
2022/01. Enero
2022/02. Febrero
2022/03. Marzo
2022/05. Mayo
2022/11. Noviembre
2022/12. Diciembre
2023/01. Enero
2023/02. Febrero
2023/03. Marzo
2023/04. Abril
2023/05. Mayo
2023/06. Junio
2023/07. Julio
2023/08. Agosto
2023/09. Septiembre
2023/10. Octubre
2023/11. Noviembre
2023/12. Diciembre
2024/01. Enero
2024/02. Febrero
2024/03. Marzo
2024/04. Abril
2024/05. Mayo
2024/06. Junio
2024/07. Julio
2024/08. Agosto
2024/09. Septiembre
2024/10. Octubre
2024/11. Noviembre
2024/12. Diciembre
2025/01. Enero
2025/02. Febrero
2025/03. Marzo


,Año,Mes,Referencia Factura
1196,2024,01. Enero,9300304277
1197,2024,01. Enero,9300304278
1198,2024,01. Enero,9300304279
1199,2024,01. Enero,9300304280
1200,2024,01. Enero,9300304329
...,...,...,...
1988,2025,03. Marzo,9300364865
1989,2025,03. Marzo,9300364869
1990,2025,03. Marzo,9300364870
1991,2025,03. Marzo,9300364891


In [7]:
path_facturas_cliente

'c:\\Users\\mikel.perez\\0_Python_projects_v2\\data\\customers\\elec/PR/facturas/'

In [8]:
len(df_keys_fil[(df_keys_fil['Mes'] == '02. Febrero') & (df_keys_fil['Año'] == 2025)]['Referencia Factura'].tolist())

31

In [9]:
df_alias = pd.read_excel(path_alias_elec)
print((df_alias.iloc[4,2]))
df_alias

ES0021000001562230MK


,cliente,alias,cups,tarifa,electrointensivo,contrato
0,PR,DELFINARIO,ES0031408040511001BG,3.0 TD,False,1
1,PR,AQUAPARK,ES0031405931106001DF,3.0 TD,False,1
2,PR,CARTAYA,ES0031101483796004QM,6.1 TD,False,1
3,PR,CULLERA,ES0021000007935634YX,6.1 TD,False,1
4,PR,TORREVIEJA,ES0021000001562230MK,6.1 TD,False,1
...,...,...,...,...,...,...
105,Catalonia Fundacio,EDDA DE CARLI,ES0031408015307001CW,2.0 TD,False,1
106,Catalonia Fundacio,BESORA,ES0031405265052002WH,2.0 TD,False,1
107,Catalonia Fundacio,S JORDI 2,ES0031405060747021FN,2.0 TD,False,1
108,Catalonia Fundacio,HOSPITALET,ES0031406166448011FZ,2.0 TD,False,1


In [10]:
import pandas as pd
from sqlalchemy import create_engine

engine = create_engine(f'sqlite:///{db_elec_path}', echo=True)

In [11]:
try:
    existing_data = pd.read_sql(('SELECT * FROM facturas'), con=engine)
    compiled_fras = existing_data['num_factura'].tolist()
except Exception as e:
    print(f"Error occurred or table does not exist: {e}")
    existing_data = pd.DataFrame()  
    compiled_fras = []

2025-07-03 08:33:13,443 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-03 08:33:13,445 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("SELECT * FROM facturas")
2025-07-03 08:33:13,447 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-03 08:33:13,450 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("SELECT * FROM facturas")
2025-07-03 08:33:13,450 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-03 08:33:13,452 INFO sqlalchemy.engine.Engine SELECT * FROM facturas
2025-07-03 08:33:13,456 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-03 08:33:13,568 INFO sqlalchemy.engine.Engine ROLLBACK


In [12]:
df = pd.DataFrame()

facturas_list = os.listdir(os.path.abspath(f'{path_fras_elec_xml_clean}'))
facturas = []

for fra in ((facturas_list)):
    
    if (("xml" in fra or "XML" in fra) and (fra[:-4] not in compiled_fras)):

        print (f'Factura-e: {fra}')
        path_factura = os.path.abspath(f'{path_fras_elec_xml_clean}/{fra}')
        conceptos_factura = data_extraction(path_factura)
        facturas.append(conceptos_factura)
        print()
        facturas_df = pd.DataFrame(facturas)
        df = pd.concat([df, facturas_df], ignore_index=True)

df.drop_duplicates(subset='num_factura', inplace=True)
df = df[~df['num_factura'].str.contains('VARIOS', case=False, na=False)]
df['cliente'] = df.apply(lambda row: insert_alias(row['cups'], df_alias)[0], axis=1)
df['alias'] = df.apply(lambda row: insert_alias(row['cups'], df_alias)[1], axis=1)

Factura-e: VARIOS_2300007667.xml
TOTALENERGIES ELECTRICIDAD Y G
Inicio periodo: 0001-01-01
Fin periodo: 9999-12-31
Fecha emision factura: 2023-03-29
CUPS: ES0031405062297001FE

Potencia contratada por periodo:
[140.0, 140.0, 140.0, 140.0, 140.0, 200.0]

Energia consumida por periodo:
[0, 0, 0, 0, 0, 0]

Coste potencia: 0 €
Coste energia: -72.95 €
Excesos de potencia: 0.0
Descuento electrointensivos: 0.0 €
Coste reactiva: 0.0 €
Total bruto (sin I.E ni IVA)  de potencia: -72.95 €
Base imponible (sin IVA): -72.95 €

Factura-e: VARIOS_2300008465.xml
TOTALENERGIES ELECTRICIDAD Y G
Inicio periodo: 0001-01-01
Fin periodo: 9999-12-31
Fecha emision factura: 2023-03-29
CUPS: ES0031408107624001AQ

Potencia contratada por periodo:
[15.0, 0.0, 15.0, 0, 0, 0]

Energia consumida por periodo:
[0, 0, 0, 0, 0, 0]

Coste potencia: 0 €
Coste energia: -7.06 €
Excesos de potencia: 0.0
Descuento electrointensivos: 0.0 €
Coste reactiva: 0.0 €
Total bruto (sin I.E ni IVA)  de potencia: -7.06 €
Base imponible (

In [13]:
if not len(df) == 0: 
    for column in ['inicio_periodo', 'fin_periodo', 'issue_date']:
        df[column] = pd.to_datetime(df[column])
        # df[column] = df[column].dt.date
    potencia_df = pd.DataFrame(df['potencia'].tolist(), index=df.index)
    potencia_df.columns = [f'potencia_p{i+1}' for i in range(potencia_df.shape[1])]
    energia_df = pd.DataFrame(df['energia'].tolist(), index=df.index)
    energia_df.columns = [f'energia_p{i+1}' for i in range(energia_df.shape[1])]
    maximetros_df = pd.DataFrame(df['maximetros'].tolist(), index=df.index)
    maximetros_df.columns = [f'maximetros_p{i+1}' for i in range(maximetros_df.shape[1])]
    df_facturas = pd.concat([df, potencia_df, energia_df, maximetros_df], axis=1)
    df_facturas = df_facturas.drop(columns=['potencia', 'energia', 'maximetros'])
    df_facturas = df_facturas.drop_duplicates(subset='num_factura')
    df_facturas['year'] = df_facturas['inicio_periodo'].dt.year
    df_facturas['month'] = df_facturas['inicio_periodo'].dt.month
    # df_expanded = df_expanded.groupby(['year','month', 'cups']).sum(numeric_only=True).reset_index()
    # df_expanded = df_expanded.groupby(['year','month']).sum(numeric_only=True).reset_index()
    df_facturas = df_facturas.round(2)
    # print(df_facturas.tail())

    columns = list(df_facturas.columns[5:7]) + list(df_facturas.columns[9:12]) + (list(df_facturas.columns[14:32]))
    print(df_facturas.columns[8])

    def enforce_negative(df: pd.DataFrame, column_list):
        # Select rows where 'num_factura' starts with 'FELEC'
        mask_felec = df['num_factura'].str.contains('ELE', na=False)

        # Within those rows, find rows with negative values in specified columns
        mask_negative = (df.loc[mask_felec, column_list] < 0).any(axis=1)

        # Get the indices of rows matching both conditions
        indices_to_modify = df.loc[mask_felec].index[mask_negative]

        # Enforce negative values
        df.loc[indices_to_modify, column_list] = df.loc[indices_to_modify, column_list].abs() * -1

        return df

    df_facturas = enforce_negative(df_facturas, columns)
else:
    df_facturas = existing_data


In [14]:
if not existing_data.empty:
    new_rows = df_facturas[~df_facturas['num_factura'].isin(existing_data['num_factura'])]
else:
    new_rows = df_facturas
if not new_rows.empty:
    new_rows.to_sql('facturas', con=engine, if_exists='append', index=False)
    print(f"{len(new_rows)} new rows have been added to the database.")
else:
    print("No new rows to add; all rows already exist in the database.")


No new rows to add; all rows already exist in the database.


In [15]:
df_facturas = pd.read_sql(('SELECT * FROM facturas'), con=engine)

2025-07-03 08:33:18,250 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-03 08:33:18,252 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("SELECT * FROM facturas")
2025-07-03 08:33:18,254 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-03 08:33:18,256 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("SELECT * FROM facturas")
2025-07-03 08:33:18,257 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-03 08:33:18,259 INFO sqlalchemy.engine.Engine SELECT * FROM facturas
2025-07-03 08:33:18,261 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-03 08:33:18,348 INFO sqlalchemy.engine.Engine ROLLBACK


In [16]:
df_facturas['cliente'].unique()
df_facturas[df_facturas['alias'] == 'EDIFICI 1']

,num_factura,inicio_periodo,fin_periodo,issue_date,cups,coste_potencia,coste_energia,excesos_potencia,dto_electrointensivo,coste_reactiva,...,energia_p5,energia_p6,maximetros_p1,maximetros_p2,maximetros_p3,maximetros_p4,maximetros_p5,maximetros_p6,year,month
967,FELEC_2300009756,2022-12-01 00:00:00.000000,2022-12-31 00:00:00.000000,2023-01-11 00:00:00.000000,ES0031405062297002PT,266.80,5150.29,0.0,0.0,0.00,...,0.0,9566.0,52.0,52.0,0.0,0.0,0.0,48.0,2022,12
1009,FELEC_2300042616,2023-01-01 00:00:00.000000,2023-01-31 00:00:00.000000,2023-02-07 00:00:00.000000,ES0031405062297002PT,260.04,2598.87,0.0,0.0,0.00,...,0.0,9727.0,64.0,60.0,0.0,0.0,0.0,52.0,2023,1
1048,FELEC_2300078960,2023-02-01 00:00:00.000000,2023-02-28 00:00:00.000000,2023-03-06 00:00:00.000000,ES0031405062297002PT,234.86,3123.91,0.0,0.0,0.00,...,0.0,7611.0,56.0,48.0,0.0,0.0,0.0,48.0,2023,2
1086,FELEC_2300116591,2023-03-01 00:00:00.000000,2023-03-31 00:00:00.000000,2023-04-05 00:00:00.000000,ES0031405062297002PT,260.04,1980.01,0.0,0.0,0.00,...,0.0,6144.0,0.0,48.0,40.0,0.0,0.0,36.0,2023,3
1141,FELEC_2300169531,2023-04-01 00:00:00.000000,2023-04-30 00:00:00.000000,2023-05-09 00:00:00.000000,ES0031405062297002PT,251.65,1276.42,0.0,0.0,0.00,...,1682.0,5405.0,0.0,0.0,0.0,40.0,32.0,40.0,2023,4
1179,FELEC_2300213773,2023-05-01 00:00:00.000000,2023-05-31 00:00:00.000000,2023-06-09 00:00:00.000000,ES0031405062297002PT,260.04,1127.75,0.0,0.0,0.00,...,1796.0,4793.0,0.0,0.0,0.0,36.0,28.0,36.0,2023,5
1239,FELEC_2300284189,2023-06-01 00:00:00.000000,2023-06-30 00:00:00.000000,2023-07-18 00:00:00.000000,ES0031405062297002PT,251.65,2186.46,0.0,0.0,15.21,...,0.0,6925.0,0.0,0.0,72.0,64.0,0.0,76.0,2023,6
1273,FELEC_2300308741,2023-07-01 00:00:00.000000,2023-07-31 00:00:00.000000,2023-08-08 00:00:00.000000,ES0031405062297002PT,260.04,4017.24,0.0,0.0,182.63,...,0.0,12644.0,80.0,72.0,0.0,0.0,0.0,104.0,2023,7
1317,FELEC_2300360189,2023-08-01 00:00:00.000000,2023-08-31 00:00:00.000000,2023-09-08 00:00:00.000000,ES0031405062297002PT,260.04,3817.30,0.0,0.0,164.62,...,0.0,13783.0,0.0,0.0,80.0,72.0,0.0,76.0,2023,8
1353,FELEC_2300410524,2023-09-01 00:00:00.000000,2023-09-30 00:00:00.000000,2023-10-09 00:00:00.000000,ES0031405062297002PT,251.65,2421.51,0.0,0.0,0.00,...,0.0,8182.0,0.0,0.0,68.0,60.0,0.0,80.0,2023,9


In [17]:
df_facturas.sort_values(by='issue_date', ascending=False)

,num_factura,inicio_periodo,fin_periodo,issue_date,cups,coste_potencia,coste_energia,excesos_potencia,dto_electrointensivo,coste_reactiva,...,energia_p5,energia_p6,maximetros_p1,maximetros_p2,maximetros_p3,maximetros_p4,maximetros_p5,maximetros_p6,year,month
2461,FELEC_2500439872,2025-05-01 00:00:00.000000,2025-05-31 00:00:00.000000,2025-06-25 00:00:00.000000,ES0021000004106053EK,635.03,1361.47,0.0,0.0,0.0,...,4321.00,8554.00,0.0,0.00,0.00,126.0,114.0,92.00,2025,5
2453,FELEC_2500439118,2025-05-01 00:00:00.000000,2025-05-31 00:00:00.000000,2025-06-24 00:00:00.000000,ES0021000004832397RW,52.93,59.54,0.0,0.0,0.0,...,315.00,128.00,0.0,0.00,0.00,14.0,9.0,1.00,2025,5
2447,FELEC_2500439108,2025-05-01 00:00:00.000000,2025-05-31 00:00:00.000000,2025-06-24 00:00:00.000000,ES0031406105718001HD,424.31,1588.04,0.0,0.0,0.0,...,5832.43,5402.05,0.0,0.00,0.00,104.0,96.0,76.00,2025,5
2460,FELEC_2500439127,2025-05-19 00:00:00.000000,2025-06-16 00:00:00.000000,2025-06-24 00:00:00.000000,ES0031405525900001NA,7.57,7.87,0.0,0.0,0.0,...,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.00,2025,5
2459,FELEC_2500439126,2025-05-02 00:00:00.000000,2025-06-01 00:00:00.000000,2025-06-24 00:00:00.000000,ES0022000007202761WE,32.51,37.16,0.0,0.0,0.0,...,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.00,2025,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
955,FELEC_2300001711,2022-12-01 00:00:00.000000,2022-12-30 00:00:00.000000,2023-01-04 00:00:00.000000,ES0031408108209001TB,22.44,1.92,0.0,0.0,0.0,...,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.00,2022,12
956,FELEC_2300001743,2022-12-01 00:00:00.000000,2022-12-30 00:00:00.000000,2023-01-04 00:00:00.000000,ES0031408107624001AQ,33.67,85.98,0.0,0.0,0.0,...,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.00,2022,12
957,FELEC_2300001758,2022-11-29 00:00:00.000000,2022-12-30 00:00:00.000000,2023-01-04 00:00:00.000000,ES0031408015307001CW,23.93,241.25,0.0,0.0,0.0,...,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.00,2022,11
958,FELEC_2300001817,2022-12-01 00:00:00.000000,2022-12-31 00:00:00.000000,2023-01-04 00:00:00.000000,ES0031408051223001TF,260.12,6615.72,0.0,0.0,0.0,...,0.00,13152.00,56.0,60.00,0.00,0.0,0.0,64.00,2022,12


In [23]:
df_facturas[df_facturas['alias'] == 'KEM ONE'].sort_values(by=['year', 'month', 'issue_date'], ascending=False)

,num_factura,inicio_periodo,fin_periodo,issue_date,cups,coste_potencia,coste_energia,excesos_potencia,dto_electrointensivo,coste_reactiva,...,energia_p5,energia_p6,maximetros_p1,maximetros_p2,maximetros_p3,maximetros_p4,maximetros_p5,maximetros_p6,year,month
2438,FELEC_2500435520,2025-05-01 00:00:00.000000,2025-05-31 00:00:00.000000,2025-06-20 00:00:00.000000,ES0021000003225511XB,0.00,42705.32,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2025,5
2392,ABOELE_2500006539,2025-04-01 00:00:00.000000,2025-04-30 00:00:00.000000,2025-06-09 00:00:00.000000,ES0021000003225511XB,0.00,237450.92,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2025,4
2375,ABOELE_2500006109,2025-04-01 00:00:00.000000,2025-04-30 00:00:00.000000,2025-05-26 00:00:00.000000,ES0021000003225511XB,0.00,-237450.92,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2025,4
2386,FRELEC_2500004306,2025-04-01 00:00:00.000000,2025-04-30 00:00:00.000000,2025-05-26 00:00:00.000000,ES0021000003225511XB,0.00,39324.86,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2025,4
953,ABOELE_2500005525,2025-04-01 00:00:00.000000,2025-04-30 00:00:00.000000,2025-05-13 00:00:00.000000,ES0021000003225511XB,0.00,-237450.92,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2025,4
2225,FELEC_2500324906,2025-04-01 00:00:00.000000,2025-04-30 00:00:00.000000,2025-05-13 00:00:00.000000,ES0021000003225511XB,0.00,237450.92,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2025,4
2226,FELEC_2500324907,2025-03-01 00:00:00.000000,2025-03-31 00:00:00.000000,2025-05-13 00:00:00.000000,ES0021000003225511XB,0.00,74566.24,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2025,3
2156,FELEC_2500201731,2025-02-01 00:00:00.000000,2025-02-28 00:00:00.000000,2025-03-14 00:00:00.000000,ES0021000003225511XB,0.00,90191.73,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2025,2
2052,FELEC_2500045265,2024-12-01 00:00:00.000000,2024-12-31 00:00:00.000000,2025-01-15 00:00:00.000000,ES0021000003225511XB,0.00,75630.58,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2024,12
2017,FELEC_2400859971,2024-11-01 00:00:00.000000,2024-11-30 00:00:00.000000,2024-12-18 00:00:00.000000,ES0021000003225511XB,0.00,62003.10,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2024,11


In [19]:
df_facturas[df_facturas['cliente'] == 'Altan Pharma'].sort_values(by=['year', 'month', 'issue_date'], ascending=False)

,num_factura,inicio_periodo,fin_periodo,issue_date,cups,coste_potencia,coste_energia,excesos_potencia,dto_electrointensivo,coste_reactiva,...,energia_p5,energia_p6,maximetros_p1,maximetros_p2,maximetros_p3,maximetros_p4,maximetros_p5,maximetros_p6,year,month
2446,FELEC_2500438968,2025-05-01 00:00:00.000000,2025-05-31 00:00:00.000000,2025-06-24 00:00:00.000000,ES0022000008830951EL,1787.13,12843.42,0.00,0.0,838.72,...,39378.0,92636.0,0.00,0.00,0.00,0.00,0.00,0.00,2025,5
2451,FELEC_2500439113,2025-05-01 00:00:00.000000,2025-05-31 00:00:00.000000,2025-06-24 00:00:00.000000,ES0021000000014120VH,146.39,752.49,0.00,0.0,3.13,...,2114.0,4526.0,0.00,0.00,0.00,26.00,28.00,22.00,2025,5
2455,FELEC_2500439122,2025-05-01 00:00:00.000000,2025-05-31 00:00:00.000000,2025-06-24 00:00:00.000000,ES0021000003982535BJ,490.78,714.38,0.00,0.0,0.00,...,2353.0,3523.0,0.00,0.00,0.00,66.00,68.00,60.00,2025,5
2394,FELEC_2500427019,2025-05-01 00:00:00.000000,2025-05-31 00:00:00.000000,2025-06-16 00:00:00.000000,ES0021000000014118VQ,1623.80,9780.10,7.98,0.0,0.00,...,34043.0,61394.0,0.00,0.00,0.00,347.00,339.00,275.00,2025,5
2388,FELEC_2500390202,2025-05-04 00:00:00.000000,2025-05-31 00:00:00.000000,2025-06-06 00:00:00.000000,ES0022000008689924DM,29.36,4.75,0.00,0.0,0.00,...,0.0,0.0,1.13,0.00,0.70,0.00,0.00,0.00,2025,5
2390,FELEC_2500390853,2025-05-01 00:00:00.000000,2025-05-31 00:00:00.000000,2025-06-06 00:00:00.000000,ES0022000008212349ZW,52.97,225.30,2.64,0.0,0.00,...,1143.0,390.0,0.00,0.00,0.00,17.38,17.64,17.91,2025,5
2227,FELEC_2500336043,2025-04-01 00:00:00.000000,2025-04-30 00:00:00.000000,2025-05-21 00:00:00.000000,ES0021000003982535BJ,474.95,553.01,0.00,0.0,0.00,...,1463.0,3784.0,0.00,0.00,0.00,52.00,51.00,47.00,2025,4
2228,FELEC_2500336044,2025-04-01 00:00:00.000000,2025-04-30 00:00:00.000000,2025-05-21 00:00:00.000000,ES0021000000014120VH,141.68,666.36,0.00,0.0,18.71,...,1875.0,4185.0,0.00,0.00,0.00,25.00,23.00,24.00,2025,4
2229,FELEC_2500336045,2025-04-01 00:00:00.000000,2025-04-30 00:00:00.000000,2025-05-21 00:00:00.000000,ES0021000000014118VQ,1571.44,7631.16,0.00,0.0,0.00,...,27745.0,51441.0,0.00,0.00,0.00,287.00,309.00,303.00,2025,4
2230,FELEC_2500336046,2025-04-01 00:00:00.000000,2025-04-30 00:00:00.000000,2025-05-21 00:00:00.000000,ES0022000008212349ZW,51.24,213.86,4.36,0.0,0.00,...,1093.0,319.0,0.00,0.00,0.00,20.30,19.81,15.22,2025,4


In [20]:
columns = list(df_facturas.columns[5:7]) + list(df_facturas.columns[9:12]) + (list(df_facturas.columns[14:32]))
columns

['coste_potencia',
 'coste_energia',
 'coste_reactiva',
 'total_bruto_ie_iva',
 'total_bruto_iva',
 'potencia_p1',
 'potencia_p2',
 'potencia_p3',
 'potencia_p4',
 'potencia_p5',
 'potencia_p6',
 'energia_p1',
 'energia_p2',
 'energia_p3',
 'energia_p4',
 'energia_p5',
 'energia_p6',
 'maximetros_p1',
 'maximetros_p2',
 'maximetros_p3',
 'maximetros_p4',
 'maximetros_p5',
 'maximetros_p6']

In [21]:
alias_list = df_alias[df_alias['cliente'] == 'Grupo Inspired']['alias'].to_list()
alias_list
df_facturas_fil = df_facturas[df_facturas['cliente'] == 'Grupo Inspired']
df_grupo_inspired = pd.DataFrame()

for alias in alias_list:

    df_facturas_list = df_facturas_fil[df_facturas_fil['alias'] == alias]
    df_facturas_list.sort_values(by='fin_periodo', ascending=False, inplace=True)
    if df_grupo_inspired.empty:
        df_grupo_inspired = df_facturas_list.head(1)
    else:
        df_grupo_inspired = pd.concat([df_grupo_inspired, df_facturas_list.head(1)], ignore_index=True)

df_grupo_inspired.to_excel('gri_last_fra.xlsx')

C:\Users\mikel.perez\AppData\Local\Temp\ipykernel_25624\3654977364.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_facturas_list.sort_values(by='fin_periodo', ascending=False, inplace=True)


In [22]:
df_cf = df_facturas[(df_facturas['cliente'] == 'Catalonia Fundacio') & (df_facturas['year'] == 2024)]
df_cf['tarifa'] = df_cf['alias'].apply(lambda x: df_alias[df_alias['alias'] == x]['tarifa'].values[0])
df_cf.groupby(['month', 'tarifa'])[[f'energia_p{i}' for i in range (1,7)] + ['coste_potencia'] + ['excesos_potencia']].sum().reset_index().to_excel('fundacio_cat.xlsx')

C:\Users\mikel.perez\AppData\Local\Temp\ipykernel_25624\609581090.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cf['tarifa'] = df_cf['alias'].apply(lambda x: df_alias[df_alias['alias'] == x]['tarifa'].values[0])
